In [1]:
from itertools import product
import numpy as np
import networkx as nx
import heapq
import pandas as pd
import time

In [2]:
def create_problem(size: int, density: float = 1.0, negative_values: bool = False, noise_level: float = 0.0, seed: int = 42) -> np.ndarray:
    rng = np.random.default_rng(seed)
    map = rng.random(size=(size, 2))
    problem = rng.random((size, size))
    if negative_values:
        problem = problem * 2 - 1
    problem *= noise_level
    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            problem[a, b] += np.sqrt(np.square(map[a, 0] - map[b, 0]) + np.square(map[a, 1] - map[b, 1]))
        else:
            problem[a, b] = np.inf
    np.fill_diagonal(problem, 0)
    return (problem * 1_000).round()

In [3]:
def build_graph(matrix):
    # Creates a NetworkX DiGraph from the matrix for structure only.
    masked = np.ma.masked_array(matrix, mask=np.isinf(matrix))
    G = nx.from_numpy_array(masked, create_using=nx.DiGraph)
    return G

def reconstruct_path(predecessors, start, end):
    # Backtracks to rebuild the path list.
    if end not in predecessors and start != end: return None
    path = [end]
    curr = end
    while curr != start:
        curr = predecessors.get(curr)
        if curr is None: return None
        path.append(curr)
    return path[::-1]

In [4]:
def run_dijkstra(G, start, end):
    # Dijkstra's Algorithm
    distances = {node: float('inf') for node in G.nodes}
    distances[start] = 0
    predecessors = {}
    pq = [(0, start)]
    visited = set()
    
    while pq:
        d, u = heapq.heappop(pq)
        if u == end: break
        if u in visited: continue
        visited.add(u)
        
        for v, data in G[u].items():
            weight = data.get('weight')
            if weight < 0: return None, "Error: Neg Weight"
            if distances[u] + weight < distances[v]:
                distances[v] = distances[u] + weight
                predecessors[v] = u
                heapq.heappush(pq, (distances[v], v))
                
    path = reconstruct_path(predecessors, start, end)
    return path, distances[end]

def run_astar(G, start, end):
    # A* Algorithm
    def heuristic(n): return 0 
    g_score = {node: float('inf') for node in G.nodes}
    g_score[start] = 0
    predecessors = {}
    pq = [(0, start)]
    visited = set()
    
    while pq:
        _, u = heapq.heappop(pq)
        if u == end: break
        if u in visited: continue
        visited.add(u)
        
        for v, data in G[u].items():
            weight = data.get('weight')
            if weight < 0: return None, "Error: Neg Weight"
            
            tentative = g_score[u] + weight
            if tentative < g_score[v]:
                g_score[v] = tentative
                predecessors[v] = u
                heapq.heappush(pq, (tentative + heuristic(v), v))
                
    path = reconstruct_path(predecessors, start, end)
    return path, g_score[end]

def run_bellman_ford(G, start, end):
    # Bellman-Ford
    distances = {node: float('inf') for node in G.nodes}
    distances[start] = 0
    predecessors = {}
    edges = list(G.edges(data=True))
    num_nodes = len(G.nodes)
    
    # Relaxation
    for _ in range(num_nodes - 1):
        changed = False
        for u, v, data in edges:
            w = data['weight']
            if distances[u] != float('inf') and distances[u] + w < distances[v]:
                distances[v] = distances[u] + w
                predecessors[v] = u
                changed = True
        if not changed: break
        
    # Neg Cycle Check
    for u, v, data in edges:
        w = data['weight']
        if distances[u] != float('inf') and distances[u] + w < distances[v]:
            return None, "Error: Neg Cycle"
            
    path = reconstruct_path(predecessors, start, end)
    return path, distances[end]

In [5]:
sizes = [10, 20, 50, 100, 200, 500] 
densities = [0.2, 0.5, 0.8, 1.0]
noises = [0.0, 0.1, 0.5, 0.8]
neg_opts = [False, True]

results = []

print(f"Running Experiments... (This may take 1-2 mins)")

for size in sizes:
    start_n, end_n = 0, size - 1
    for density in densities:
        for noise in noises:
            for is_neg in neg_opts:
                
                # 1. Generate
                matrix = create_problem(size, density=density, negative_values=is_neg, noise_level=noise)
                G = build_graph(matrix)
                
                # 2. Select Algos
                if is_neg:
                    current_algos = [("Bellman-Ford", run_bellman_ford)]
                else:
                    current_algos = [("Dijkstra", run_dijkstra), ("A*", run_astar), ("Bellman-Ford", run_bellman_ford)]
                
                # 3. Run
                for name, func in current_algos:
                    t0 = time.perf_counter()
                    path, cost = func(G, start_n, end_n)
                    t1 = time.perf_counter()
                    
                    # 4. Format Result
                    cost_str = cost
                    if isinstance(cost, (int, float)) and cost != float('inf'):
                        cost_str = round(cost, 2)
                    elif cost == float('inf'):
                        cost_str = "Inf"
                        
                    results.append({
                        "Size": size,
                        "Density": density,
                        "Noise": noise,
                        "Is_Negative": is_neg,
                        "Algorithm": name,
                        "Cost": cost_str,
                        "Time_ms": round((t1 - t0) * 1000, 4),
                        "Path": str(path) if path else "None"
                    })

Running Experiments... (This may take 1-2 mins)


In [6]:
df = pd.DataFrame(results)

print("\n" + "="*60)
print("TABLE 1: POSITIVE ENVIRONMENTS (Is_Negative = False)")
print("="*60)
# Filter for Positive environments and Valid paths
pos_df = df[(df['Is_Negative'] == False) & (df['Path'] != 'None')]
# Show a diverse sample
cols = ['Size', 'Density', 'Noise', 'Algorithm', 'Cost', 'Time_ms']
print(pos_df[cols].groupby('Size').apply(lambda x: x.iloc[::4]).to_string(index=False))
# Note: .iloc[::4] skips rows to show you a mix of densities (0.2, 1.0) and noises, not just the first one.

print("\n" + "="*60)
print("TABLE 2: NEGATIVE ENVIRONMENTS (Is_Negative = True)")
print("="*60)
neg_df = df[df['Is_Negative'] == True]
# Show successful paths (where cost is a number, even if negative)
valid_neg_df = neg_df[neg_df['Path'] != 'None']

if not valid_neg_df.empty:
    print(valid_neg_df[cols].groupby('Size').apply(lambda x: x.iloc[::4]).to_string(index=False))
else:
    print("No valid paths found in negative environments (Likely Negative Cycles or Unreachable).")
    # Show errors if no paths found
    print("\nSample Errors:")
    print(neg_df[['Size', 'Cost']].head())


TABLE 1: POSITIVE ENVIRONMENTS (Is_Negative = False)
 Size  Density  Noise    Algorithm    Cost  Time_ms
   10      0.2    0.0     Dijkstra  1676.0   0.0413
   10      0.2    0.1           A*  1787.0   0.0159
   10      0.2    0.5 Bellman-Ford  2231.0   0.0204
   10      0.5    0.0     Dijkstra   670.0   0.0179
   10      0.5    0.1           A*   767.0   0.0134
   10      0.5    0.5 Bellman-Ford  1155.0   0.0285
   10      0.8    0.0     Dijkstra   200.0   0.0106
   10      0.8    0.1           A*   268.0   0.0065
   10      0.8    0.5 Bellman-Ford   542.0   0.0409
   10      1.0    0.0     Dijkstra   200.0   0.0113
   10      1.0    0.1           A*   268.0   0.0065
   10      1.0    0.5 Bellman-Ford   542.0   0.0448
   20      0.2    0.0     Dijkstra  1124.0   0.0963
   20      0.2    0.1           A*  1232.0   0.0303
   20      0.2    0.5 Bellman-Ford  1664.0   0.0501
   20      0.5    0.0     Dijkstra   704.0   0.0464
   20      0.5    0.1           A*   783.0   0.0442
   20     